In [1]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import matplotlib.pyplot as plt
import os

2024-04-21 13:40:21.122319: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 13:40:21.122415: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 13:40:21.250424: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
from pathlib import Path

In [5]:
from tensorflow.keras.applications import VGG19

Set GPU Instance

In [6]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Parameters

In [7]:
AUGMENTATION_TYPE = 'rand_aug'
# [no, rand_aug]
IMAGE_SIZE = (224,224)
BATCH_SIZE = 32
INCLUDE_TOP = False
EPOCHS = 100
NUM_CLASSES = 8
CLASS_MODE = 'categorical'
WEIGHTS = 'imagenet'
PATIENCE = 20

Train and Test Datagen preparation

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=20,  # Random rotation
    width_shift_range=0.2,  # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,  # Shear transformation
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'
)

In [9]:
val_datagen = ImageDataGenerator(
    rescale=1./255
)

In [10]:
train_dir = '/kaggle/input/isic-2019/ISIC - 2019/train'
val_dir = '/kaggle/input/isic-2019/ISIC - 2019/val'

In [11]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    class_mode = CLASS_MODE,
    shuffle = True
)

Found 21491 images belonging to 8 classes.


In [12]:
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    class_mode = CLASS_MODE,
    shuffle = False
)

Found 1910 images belonging to 8 classes.


VGG19_Training Instance

In [13]:
class vgg19Training:
    def __init__(self):
        self.early_stopping = EarlyStopping(monitor='val_loss',patience=20,restore_best_weights=True)

    def define_base_model(self):
        base_model = VGG19(weights='imagenet', include_top=False)

        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(256, activation='relu')(x)
        predictions = Dense(8, activation='softmax')(x)

        model = Model(inputs=base_model.input, outputs=predictions)

        for layer in base_model.layers:
            layer.trainable = False

        print(model.summary())
        
        return model
    
    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model_path = os.path.join(path,'vgg19.h5')
        model.save(model_path)
    
    def get_model_and_compile(self):
        with tf.device('/GPU:0'):
            self.vgg19_model = self.define_base_model()
            self.vgg19_model.compile(
                optimizer = 'adam',
                loss = "categorical_crossentropy",
                metrics = ['accuracy']
            )
    
    def train_model(self,train_generator,val_generator):
        with tf.device('/GPU:0'):
            self.history = self.vgg19_model.fit(
                train_generator,
                steps_per_epoch = len(train_generator),
                epochs = 100,
                validation_data = val_generator,
                validation_steps = len(val_generator),
                callbacks = [self.early_stopping]
            )

        self.save_model(
            path='/kaggle/working/',
            model=self.vgg19_model
        )

VGG19 Training Pipeline

In [14]:
try:
    vgg_trainer = vgg19Training()
    vgg_trainer.get_model_and_compile()
    vgg_trainer.train_model(train_generator=train_generator,val_generator=val_generator)

except Exception as e:
    raise e


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, None, None, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, None, None, 64) │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, None, None, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, None, None,     │        73,856 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, None, None,     │       147,584 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, None, None,     │       295,168 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv4 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, None, None,     │     1,180,160 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv4 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │             

 Total params: 20,157,768 (76.90 MB)

 Trainable params: 133,384 (521.03 KB)

...

424/672 ━━━━━━━━━━━━━━━━━━━━ 3:27 839ms/step - accuracy: 0.5125 - loss: 1.4147

2024-04-21 13:47:12.740265: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 3.83445, expected 3.08393
2024-04-21 13:47:12.740327: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 5.81387, expected 5.06335
2024-04-21 13:47:12.740336: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 5.65478, expected 4.90426
2024-04-21 13:47:12.740345: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 6.2384, expected 5.48788
2024-04-21 13:47:12.740352: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 5.85899, expected 5.10847
2024-04-21 13:47:12.740360: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 5.234, expected 4.48348
2024-04-21 13:47:12.740368: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 5.17387, expected 4.42335
2024-04-21 13:47:12.740375: E external/local_xla/xla/servi

672/672 ━━━━━━━━━━━━━━━━━━━━ 0s 843ms/step - accuracy: 0.5218 - loss: 5.8220

2024-04-21 13:51:15.824360: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 4.29755, expected 3.68034
2024-04-21 13:51:15.824419: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 11: 4.79889, expected 4.18169
2024-04-21 13:51:15.824428: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 12: 2.82377, expected 2.20656
2024-04-21 13:51:15.824436: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 13: 4.03137, expected 3.41417
2024-04-21 13:51:15.824444: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 28: 5.11395, expected 4.49675
2024-04-21 13:51:15.824451: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 29: 4.78798, expected 4.17077
2024-04-21 13:51:15.824459: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 30: 4.85662, expected 4.23942
2024-04-21 13:51:15.824466: E external/local_xla/

672/672 ━━━━━━━━━━━━━━━━━━━━ 635s 909ms/step - accuracy: 0.5218 - loss: 5.8297 - val_accuracy: 0.5403 - val_loss: 134.3447
Epoch 2/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 0s 92us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 419s 618ms/step - accuracy: 0.5542 - loss: 5.2650 - val_accuracy: 0.5288 - val_loss: 119.8708
Epoch 4/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 0s 27us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 419s 617ms/step - accuracy: 0.5589 - loss: 13.1847 - val_accuracy: 0.5518 - val_loss: 135.1032
Epoch 6/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 0s 20us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 417s 614ms/step - accuracy: 0.5024 - loss: 35.2901 - val_accuracy: 0.5508 - val_loss: 88.2218
Epoch 8/100
672/672 ━━━━━━━━━━━━━

RestNet50 instance

In [15]:
from tensorflow.keras.applications import ResNet50

In [16]:
class RestNetTraining:
    def __init__(self):
        self.early_stopping = EarlyStopping(monitor='val_loss',patience=20,restore_best_weights=True)

    def define_base_model(self):
        base_model = ResNet50(weights='imagenet', include_top=False)

        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(256, activation='relu')(x)
        predictions = Dense(8, activation='softmax')(x)

        model = Model(inputs=base_model.input, outputs=predictions)

        for layer in base_model.layers:
            layer.trainable = False

        print(model.summary())
        
        return model
    
    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model_path = os.path.join(path,'restnet50.h5')
        model.save(model_path)
    
    def get_model_and_compile(self):
        with tf.device('/GPU:0'):
            self.restnet_model = self.define_base_model()
            self.restnet_model.compile(
                optimizer = 'adam',
                loss = "categorical_crossentropy",
                metrics = ['accuracy']
            )
    
    def train_model(self,train_generator,val_generator):
        with tf.device('/GPU:0'):
            self.history = self.restnet_model.fit(
                train_generator,
                steps_per_epoch = len(train_generator),
                epochs = 100,
                validation_data = val_generator,
                validation_steps = len(val_generator),
                callbacks = [self.early_stopping]
            )

        self.save_model(
            path='/kaggle/working/',
            model=self.restnet_model
        )

RestNet50 trainer pipeline

In [17]:
try:
    resnet_trainer = RestNetTraining()
    resnet_trainer.get_model_and_compile()
    resnet_trainer.train_model(train_generator=train_generator,val_generator=val_generator)

except Exception as e:
    raise e


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, None,      │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, None,      │      9,472 │ conv1_pad[0][0]   │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, None,      │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, None,      │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, None,      │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, None,      │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, None,      │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, None,      │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, None,      │          0 │ conv2_block1_1_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, None,      │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, None,      │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, None,      │          0 │ conv2_block1_2_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, None,      │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, None,      │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, None,      │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, None,      │      1,024 │ conv2_block1_3_c

 Total params: 24,114,312 (91.99 MB)

 Trainable params: 526,600 (2.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

None
Epoch 1/100
454/672 ━━━━━━━━━━━━━━━━━━━━ 2:13 614ms/step - accuracy: 0.5020 - loss: 1.5024

W0000 00:00:1713712008.877453      82 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


672/672 ━━━━━━━━━━━━━━━━━━━━ 450s 646ms/step - accuracy: 0.5056 - loss: 1.4888 - val_accuracy: 0.4932 - val_loss: 1.5008
Epoch 2/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 0s 267us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 416s 614ms/step - accuracy: 0.5276 - loss: 1.4014 - val_accuracy: 0.5037 - val_loss: 1.4468
Epoch 4/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 0s 38us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 417s 614ms/step - accuracy: 0.5267 - loss: 1.3996 - val_accuracy: 0.5162 - val_loss: 1.4456
Epoch 6/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 0s 37us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 413s 608ms/step - accuracy: 0.5255 - loss: 1.3948 - val_accuracy: 0.5037 - val_loss: 1.4428
Epoch 8/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 

Densnet Instance

In [18]:
from tensorflow.keras.applications import DenseNet121

In [19]:
class DenseNetTraining:
    def __init__(self):
        self.early_stopping = EarlyStopping(monitor='val_loss',patience=20,restore_best_weights=True)

    def define_base_model(self):
        base_model = DenseNet121(weights='imagenet', include_top=False)

        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(256, activation='relu')(x)
        predictions = Dense(8, activation='softmax')(x)

        model = Model(inputs=base_model.input, outputs=predictions)

        for layer in base_model.layers:
            layer.trainable = False

        print(model.summary())
        
        return model
    
    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model_path = os.path.join(path,'DenseNet121.h5')
        model.save(model_path)
    
    def get_model_and_compile(self):
        with tf.device('/GPU:0'):
            self.densenet_model = self.define_base_model()
            self.densenet_model.compile(
                optimizer = 'adam',
                loss = "categorical_crossentropy",
                metrics = ['accuracy']
            )
    
    def train_model(self,train_generator,val_generator):
        with tf.device('/GPU:0'):
            self.history = self.densenet_model.fit(
                train_generator,
                steps_per_epoch = len(train_generator),
                epochs = 100,
                validation_data = val_generator,
                validation_steps = len(val_generator),
                callbacks = [self.early_stopping]
            )

        self.save_model(
            path='/kaggle/working/',
            model=self.densenet_model
        )

DenseNet Train Pipeline

In [20]:
try:
    densenet_trainer = DenseNetTraining()
    densenet_trainer.get_model_and_compile()
    densenet_trainer.train_model(train_generator=train_generator,val_generator=val_generator)

except Exception as e:
    raise e


29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, None,      │          0 │ input_layer_2[0]… │
│ (ZeroPadding2D)     │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, None,      │      9,408 │ zero_padding2d[0… │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, None,      │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, None,      │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_1    │ (None, None,      │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, None,      │          0 │ zero_padding2d_1… │
│ (MaxPooling2D)      │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, None,      │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, None,      │          0 │ conv2_block1_0_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, None,      │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, None,      │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, None,      │          0 │ conv2_block1_1_b… │
│ (Activation)        │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, None,      │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, None,      │          0 │ pool1[0][0],      │
│ (Concatenate)       │ None, 96)         │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, None,      │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ None, 96)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, None,      │          0 │ conv2_block2_0_b… │
│ (Activation)        │ None, 96)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, None,      │     12,288 │ conv2_block2_0_r

 Total params: 7,301,960 (27.85 MB)

 Trainable params: 264,456 (1.01 MB)

 Non-trainable params: 7,037,504 (26.85 MB)

None
Epoch 1/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 475s 658ms/step - accuracy: 0.5986 - loss: 1.1460 - val_accuracy: 0.6419 - val_loss: 1.0040
Epoch 2/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 0s 345us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 409s 602ms/step - accuracy: 0.6503 - loss: 0.9666 - val_accuracy: 0.6393 - val_loss: 0.9604
Epoch 4/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 0s 34us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 415s 612ms/step - accuracy: 0.6616 - loss: 0.9198 - val_accuracy: 0.6513 - val_loss: 0.9362
Epoch 6/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 0s 44us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 424s 625ms/step - accuracy: 0.6687 - loss: 0.9022 - val_accuracy: 0.6513 - val_loss: 0.9451
Epoch 8/100
672/672 ━━━━

InceptionV3 training Instance

In [21]:
from tensorflow.keras.applications import InceptionV3

In [22]:
class InceptionTraining:
    def __init__(self):
        self.early_stopping = EarlyStopping(monitor='val_loss',patience=20,restore_best_weights=True)

    def define_base_model(self):
        base_model = InceptionV3(weights='imagenet', include_top=False)

        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(256, activation='relu')(x)
        predictions = Dense(8, activation='softmax')(x)

        model = Model(inputs=base_model.input, outputs=predictions)

        for layer in base_model.layers:
            layer.trainable = False

        print(model.summary())
        
        return model
    
    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model_path = os.path.join(path,'InceptionV3.h5')
        model.save(model_path)
    
    def get_model_and_compile(self):
        with tf.device('/GPU:0'):
            self.inception_model = self.define_base_model()
            self.inception_model.compile(
                optimizer = 'adam',
                loss = "categorical_crossentropy",
                metrics = ['accuracy']
            )
    
    def train_model(self,train_generator,val_generator):
        with tf.device('/GPU:0'):
            self.history = self.inception_model.fit(
                train_generator,
                steps_per_epoch = len(train_generator),
                epochs = 100,
                validation_data = val_generator,
                validation_steps = len(val_generator),
                callbacks = [self.early_stopping]
            )

        self.save_model(
            path='/kaggle/working/',
            model=self.inception_model
        )

Inception Pipeline

In [23]:
try:
    inception_trainer = InceptionTraining()
    inception_trainer.get_model_and_compile()
    inception_trainer.train_model(train_generator=train_generator,val_generator=val_generator)

except Exception as e:
    raise e


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, None,      │        864 │ input_layer_3[0]… │
│                     │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, None,      │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, None,      │          0 │ batch_normalizat… │
│ (Activation)        │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, None,      │      9,216 │ activation[0][0]  │
│                     │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None,      │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, None,      │          0 │ batch_normalizat… │
│ (Activation)        │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, None,      │     18,432 │ activation_1[0][… │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None,      │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, None,      │          0 │ batch_normalizat… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, None,      │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, None,      │      5,120 │ max_pooling2d[0]… │
│                     │ None, 80)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None,      │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ None, 80)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, None,      │          0 │ batch_normalizat… │
│ (Activation)        │ None, 80)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, None,      │    138,240 │ activation_3[0][… │
│                     │ None, 192)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None,      │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ None, 192)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, None,      │          0 │ batch_normalizat

 Total params: 22,329,384 (85.18 MB)

 Trainable params: 526,600 (2.01 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

None
Epoch 1/100
 74/672 ━━━━━━━━━━━━━━━━━━━━ 7:10 721ms/step - accuracy: 0.4581 - loss: 1.7870

W0000 00:00:1713721130.335243      82 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


672/672 ━━━━━━━━━━━━━━━━━━━━ 471s 668ms/step - accuracy: 0.5700 - loss: 1.2733 - val_accuracy: 0.5990 - val_loss: 1.1102
Epoch 2/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 0s 287us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 428s 631ms/step - accuracy: 0.6332 - loss: 1.0264 - val_accuracy: 0.6209 - val_loss: 1.0474
Epoch 4/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 0s 29us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 430s 632ms/step - accuracy: 0.6388 - loss: 0.9881 - val_accuracy: 0.6230 - val_loss: 1.0189
Epoch 6/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 0s 28us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 452s 665ms/step - accuracy: 0.6469 - loss: 0.9763 - val_accuracy: 0.6236 - val_loss: 1.0454
Epoch 8/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 